## Import packges

In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))

import numpy as np
import pandas as pd
import random
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
import statsmodels.api as sm

from statsmodels.graphics.gofplots import qqplot_2samples
from utils.utils import get_rt_quantiles, remove_outliers

In [2]:
sns.set_style("whitegrid");
plt.figure(figsize=(8, 6), dpi=100);

<Figure size 800x600 with 0 Axes>

In [3]:
root = "../"
behavioural_data_root = root +  "Datasets/behavioral_data/raw_data/"
data_output_path = root + "Datasets/behavioral_data/selected_data/LDT_data.csv"
qqplots_dir = "Results/Plots/qqplots/"

### Reading behavioural raw data

Reading and modifing each behavioral data file and combining all of them into a single behavioral dataframe

non-english speakers are omitted from data

participant with file number 1988 is removed from data due the problem in reading it with pandas

participant with file number 436 had two set of demogrphic data part that the dempgraphic set with the incompelete demograhic data have been removed from file

In [4]:
dataframes = []
counter = 1
for count, filename in enumerate(os.listdir(behavioural_data_root)):
    df = pd.read_csv(behavioural_data_root + filename, names=["trial", "string_id", "string_type", "accuracy", "rt", "string"])
    if df.iloc[-2, 4] != "English":
        continue
    df = df.dropna().drop("string_id", axis=1).drop([0, 1])
    df = df.iloc[:-2, :]
    ind = df.loc[df["trial"] == "Univ"].index[0]
    df = df.drop([ind, ind+1], axis=0)
    # Dropping rows with wrong accuracies
    df = df.loc[(df["accuracy"] == "0") | (df["accuracy"] == "1")]
    # Converting columns type to suitable data types
    convert_dict = {"string_type": "int16",
                    "accuracy": "int16",
                    "rt": float
                   }

    df = df.astype(convert_dict)
    # Convert RTs to seconds
    df["rt"] = df["rt"].apply(lambda x: x/1000) 
    # Removing Outliers
    df = remove_outliers(df, 3, .2, 2.5)
    # Extracting response of participant from his/her accuracy
    df["response"] = np.logical_not(np.logical_xor(df["string_type"], df["accuracy"])).astype("int")
    df = df.reset_index(drop=True)
    # Particpant number
    df["participant"] = counter
    df["quantile"]=""
    df["minRT"] = df["rt"].min()
    dataframes.append(df)
    counter += 1
behavioural_df = pd.concat(dataframes)

In [5]:
behavioural_df.head()

,trial,string_type,accuracy,rt,string,response,participant,quantile,minRT
0,1,0,0,0.548,pracker,1,1,,0.378
1,2,1,1,0.646,nearside,1,1,,0.378
2,3,1,1,0.511,jets,1,1,,0.378
3,4,0,1,0.815,vates,0,1,,0.378
4,5,1,1,0.680,onward,1,1,,0.378


C:\Users\Arash\anaconda3\envs\stan\Lib\site-packages\outdated\utils.py:14: OutdatedCheckFailedWarning: Failed to check for latest version of package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


## Participant Selection

choosing 100 random participant and checking thier mean RT and Accuracy with total mean with t-test

In [6]:
all_participants = behavioural_df.groupby(["participant"]).agg({"rt": ["mean"], "accuracy": ["mean"],
                                                                "response":["mean"]}).reset_index()
all_participants.columns = ["participant", "rt", "accuracy", "response"]

# Random selection
participants_id = random.sample(range(1, len(dataframes)), 100)
selected_participants = all_participants[all_participants["participant"].isin(participants_id)]

We want to check if our sample (100 selected participants) mean RT is seginficantly different from population (All the participants) mean RT or not?
<br>
so we use <b>One Sample T-test</b>

In [7]:
pg.ttest(selected_participants["rt"].to_numpy(), all_participants["rt"].mean())

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.385318,99,two-sided,0.700828,"[0.77, 0.84]",0.038532,0.119,0.066842


<b>There is no significant difference between selected participants RTs and all participants RT</b>

We also want to check if our sample (100 selected participants) mean Accuracy is seginficantly different from population (All the participants) mean Accuracy or not?
<br>
so we use <b>One Sample T-test</b>

In [8]:
pg.ttest(selected_participants["accuracy"].to_numpy(), all_participants["accuracy"].mean())

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.187763,99,two-sided,0.851447,"[0.85, 0.87]",0.018776,0.113,0.05397


<b>There is no significant difference between selected participants mean Accuracy and all participants mean Accuracy</b>

We also want to check if our sample (100 selected participants) mean response (choice) is seginficantly different from population (All the participants) mean response or not?
<br>
so we use <b>One Sample T-test</b>

In [9]:
pg.ttest(selected_participants["response"].to_numpy(), all_participants["response"].mean())

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.193491,99,two-sided,0.235529,"[0.47, 0.49]",0.119349,0.22,0.219114


<b>There is no significant difference between selected participants mean response (choice) and all participants mean response</b>

In [10]:
selected_participants = behavioural_df[behavioural_df["participant"].isin(participants_id)]

## Choosing 400 trials for 100 participants

Dividing data to 10 quantiles and sampling 40 random trials from each quantile for each participant

Check is done with qq-plots

In [11]:
ids = np.unique(selected_participants["participant"].to_numpy())

In [12]:
probs = np.linspace(0, 1, 400)
selected_participants_and_trials = []
with warnings.catch_warnings(record=True):
    for index, id in enumerate(ids):
        selected_participant = selected_participants[selected_participants["participant"]==id]
        selected_participant_400 = get_rt_quantiles(selected_participant, probs)

        # Quantile-quantile plot
        fig, ax = plt.subplots()
        pp_x = sm.ProbPlot(selected_participant_400["rt"])
        pp_y = sm.ProbPlot(selected_participant["rt"])
        qqplot_2samples(pp_x, pp_y, xlabel="Selected RTs Quantiles",
                        ylabel="All RTs quantules", line=None, ax=ax)
        xlim = np.linspace(*ax.get_xlim())
        ax.plot(xlim, xlim, color="orange", label="45 degree line")
        plt.title("qq plot of participant ")
        ax.legend()
        plt.title("qq plot of participant " + str(index+1))
        plt.savefig(qqplots_dir + "qq plot of participant " + str(index+1) + ".pdf")
        plt.close()
        
        selected_participants_and_trials.append(selected_participant_400)

In [13]:
final_df = pd.concat(selected_participants_and_trials)

In [14]:
final_df.groupby("string_type").count()

,trial,accuracy,rt,string,response,participant,quantile,minRT
string_type,,,,,,,,
0,19211,19211,19211,19211,19211,19211,19211,19211
1,20789,20789,20789,20789,20789,20789,20789,20789


In [15]:
final_df

,trial,string_type,accuracy,rt,string,response,participant,quantile,minRT
123,128,1,1,0.422,sand,1,3,,0.422
2159,2259,1,1,0.462,textbook,1,3,,0.422
32,34,1,1,0.481,Mets,1,3,,0.422
103,106,0,0,0.490,carmed,1,3,,0.422
2026,2119,1,1,0.503,pain,1,3,,0.422
...,...,...,...,...,...,...,...,...,...
897,1143,0,0,1.427,derminate,1,795,,0.200
1458,2056,1,0,1.446,Woodward,0,795,,0.200
873,1111,1,1,1.462,displace,1,795,,0.200
1974,2661,1,0,1.499,manes,0,795,,0.200


In [16]:
final_df = final_df.drop(["trial", "string_type", "quantile"], axis=1)
 # Adding new particpant ID column for Stan
final_df["participant_id"] = final_df["participant"].replace(ids, list(range(1, len(ids)+1)))
final_df.reset_index(inplace=True, drop=True)

In [17]:
final_df.head()

,accuracy,rt,string,response,participant,minRT,participant_id
0,1,0.422,sand,1,3,0.422,1
1,1,0.462,textbook,1,3,0.422,1
2,1,0.481,Mets,1,3,0.422,1
3,0,0.490,carmed,1,3,0.422,1
4,1,0.503,pain,1,3,0.422,1


### Saving Data

In [18]:
final_df.to_csv(data_output_path, header=0, index=False)